In [200]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
#import matplotlib.pyplot as plt
from datetime import datetime,timedelta

In [201]:
df= pd.read_csv("./data/nifty500_daily_01Jan_26Jul2023.csv")
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0',], inplace=True)

In [202]:
# Calculate the Simple Moving Average (SMA) with a window of 20,30,40,50 periods
df['SMA20'] = ta.sma(df['Close'], length=20)

df['SMA30'] = ta.sma(df['Close'], length=30)

df['SMA40'] = ta.sma(df['Close'], length=40)

df['SMA50'] = ta.sma(df['Close'], length=40)

# Calculate the Relative Strength Index (RSI) with a window of 14 periods
df['RSI'] = ta.rsi(df['Close'], length=14)


In [203]:
#df.iloc[50:65]

In [204]:
def check_symbol(input_symbol: str) -> str:
    # Check if last part is ".NS"
    if not input_symbol.endswith(".NS"):
        input_symbol += ".NS"

    # Check if the whole symbol is in capital letters
    if not input_symbol.isupper():
        input_symbol = input_symbol.upper()

    # Check if the symbol is present under the 'symbol' column of df
    if input_symbol not in df['symbol'].unique():
        raise ValueError(f"'{input_symbol}' is  not in NIFTY 500.")

    return input_symbol


In [205]:
def ma_crossover(symbol: str, band: int=5, start_date: str = None, end_date: str = None,
                 underbought: int = 50, overbought: int = 75) -> None:

    symbol = check_symbol(symbol)

    # If start_date is not provided, set it to 3 months prior to the current date
    if not start_date:
        start_date = (datetime.now() - timedelta(days=3 * 30)).strftime('%Y-%m-%d')

    # If end_date is not provided, set it to the current date
    if not end_date:
        end_date = datetime.now().strftime('%Y-%m-%d')
    # Select rows for symbol
    selected_rows = df[df['symbol'] == symbol]

    # Calculate 0.25% of the Close price
    crossover = band / 100 * selected_rows['Close']

    # Check if SMA20, SMA30, SMA40, and SMA50 are within certain Close price
    within_band = (
        (selected_rows['SMA20'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA20'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['SMA30'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA30'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['SMA40'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA40'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['SMA50'] >= (selected_rows['Close'] - crossover)) &
        (selected_rows['SMA50'] <= (selected_rows['Close'] + crossover)) &
        (selected_rows['RSI'].between(underbought, overbought)) &
        (selected_rows['date'].between(start_date, end_date))
    )

    # Filter the selected rows based on the condition
    selected_rows = selected_rows[within_band]

    # Convert the 'date' column to datetime type
    selected_rows['date'] = pd.to_datetime(selected_rows['date'])

    # Get the row with the earliest date in the selected rows
    earliest_date_row = selected_rows.iloc[selected_rows['date'].argsort()[:1]]

    # Get the row with the latest date in the selected rows
    latest_date_row = selected_rows.iloc[selected_rows['date'].argsort()[-1:]]

    # Count of selected rows
    num_selected_rows = len(selected_rows)

    # Print the count of selected rows and the resulting DataFrames
    print(f"Cross Over observed for {symbol} : {num_selected_rows} times in this period ")

    # Print the resulting DataFrames

    
    print(f"Crossover for {symbol } observed on the earliest date:")
    print(earliest_date_row[['symbol', 'Close', 'date', 'RSI']])

    print(f"\nCross over for {symbol} observed on  the latest date:")
    print(latest_date_row[['symbol', 'Close', 'date', 'RSI']])
    #print(latest_date_row.to_string(index=False))
     


In [207]:
ma_crossover('TCS')
#help(ma_crossover)

Cross Over observed for TCS.NS : 30 times in this period 
Crossover for TCS.NS observed on the earliest date:
       symbol    Close       date        RSI
46877  TCS.NS  3219.25 2023-04-28  53.948706

Cross over for TCS.NS observed on  the latest date:
       symbol    Close       date      RSI
46925  TCS.NS  3329.25 2023-07-07  60.5295


In [208]:
ma_crossover('CYIENT')

Cross Over observed for CYIENT.NS : 0 times in this period 
Crossover for CYIENT.NS observed on the earliest date:
Empty DataFrame
Columns: [symbol, Close, date, RSI]
Index: []

Cross over for CYIENT.NS observed on  the latest date:
Empty DataFrame
Columns: [symbol, Close, date, RSI]
Index: []


In [209]:
ma_crossover('ASTERDM')

Cross Over observed for ASTERDM.NS : 20 times in this period 
Crossover for ASTERDM.NS observed on the earliest date:
           symbol       Close       date        RSI
42946  ASTERDM.NS  246.949997 2023-04-28  51.485639

Cross over for ASTERDM.NS observed on  the latest date:
           symbol       Close       date        RSI
42989  ASTERDM.NS  280.200012 2023-06-30  52.631753


In [210]:
ma_crossover('COCHINSHIP')

Cross Over observed for COCHINSHIP.NS : 5 times in this period 
Crossover for COCHINSHIP.NS observed on the earliest date:
              symbol   Close       date        RSI
48682  COCHINSHIP.NS  539.75 2023-06-09  56.667278

Cross over for COCHINSHIP.NS observed on  the latest date:
              symbol  Close       date        RSI
48699  COCHINSHIP.NS  562.0 2023-07-05  51.995462
